In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from opensoundscape.datasets import SingleTargetAudioDataset
from opensoundscape.torch.train import train

/Users/tessa/Library/Caches/pypoetry/virtualenvs/opensoundscape-dxMTH98s-py3.7/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Users/tessa/Library/Caches/pypoetry/virtualenvs/opensoundscape-dxMTH98s-py3.7/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [3]:
from pathlib import Path
from pandas import DataFrame
from torchvision.models import resnet18
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import SGD
from math import floor
from sklearn.utils import shuffle

# Create a dataframe of `cardinalis-cardinalis` files

In [4]:
files = list(Path("/Volumes/seagate-storage/audio/og_files_from_10spp/").glob("**/*.mp3"))
labels = [file.parent.name for file in files]
df = DataFrame({'Destination':files, "label":labels})
df['NumericLabels'] = None
df['NumericLabels'][df.label != 'cardinalis-cardinalis'] = 0
df['NumericLabels'][df.label == 'cardinalis-cardinalis'] = 1

In [5]:
# Use this to create numeric labels for single-class, single-target training
labels_dict = {0:'cardinalis-cardinalis', 1:'other'}

# Use this to create numeric labels for multi-class, single-target training
#df.drop('filename', axis=1).drop_duplicates().values

In [6]:
#train_set_size = floor(df.size * 0.8)
train_set_size = 8
val_set_size = 2
df = shuffle(df).reset_index()
train_df = df[:train_set_size]
valid_df = df[train_set_size:train_set_size+val_set_size]

# Create a training/validation dataset
Note: this is without splitting

In [8]:
%autoreload
train_dataset = SingleTargetAudioDataset(
    df = train_df,
    label_column = 'NumericLabels',
    filename_column='Destination',
    add_noise = True, # Add noise/jitter to spectrogram
    random_trim_length = 5, #Extract a 5s clip from long files
    overlay_prob = 0.2,#with a 0.2 probability, overlay files from another class
    overlay_weight=0.2,
)
valid_dataset = SingleTargetAudioDataset(
    df = valid_df,
    label_column = 'NumericLabels',
    filename_column='Destination',
    add_noise = True, # Add noise/jitter to spectrogram
    random_trim_length = 5, #Extract a 5s clip from long files
    overlay_prob = 0.2,#with a 0.2 probability, overlay files from another class
    overlay_weight=0.2,
)

# Train the machine learning model

In [9]:
# Set up architecture for the type of model we will use
model = resnet18(pretrained = True)
model.fc = Linear(model.fc.in_features, 2)

In [10]:
save_path = Path('save_results')
if not save_path.exists(): save_path.mkdir()
out = train(
    save_dir = save_path,
    model = model,
    train_dataset = train_dataset,
    valid_dataset = valid_dataset,
    optimizer = SGD(model.parameters(), lr=1e-3),
    loss_fn = CrossEntropyLoss(),
    epochs=1,
    log_every=1,
    spec_augment=True
)